In [10]:
import numpy as np

# RoPE에서 사용할 주기적 회전 변환 함수
def apply_rotary_position_embedding(embedding, position, theta):
    """
    임베딩 벡터에 대한 회전 변환을 적용하는 함수
    :param embedding: 각 토큰의 임베딩 벡터 (numpy array)
    :param position: 토큰의 위치 (p 값)
    :param theta: 회전 각도
    :return: 회전 변환된 임베딩 벡터
    """
    embedding_rotated = np.zeros_like(embedding)
    
    cos_theta = np.cos(position * theta)
    sin_theta = np.sin(position * theta)
    
    # 회전 변환 적용: 짝수와 홀수 차원을 쌍으로 묶어 회전
    for i in range(0, len(embedding) - 1, 2):
        embedding_rotated[i] = cos_theta * embedding[i] - sin_theta * embedding[i + 1]
        embedding_rotated[i + 1] = sin_theta * embedding[i] + cos_theta * embedding[i + 1]
    
    # 홀수 차원이 남으면, 그대로 유지
    if len(embedding) % 2 == 1:
        embedding_rotated[-1] = embedding[-1]

    return embedding_rotated

# 예시 토큰 임베딩 벡터 (4개의 토큰, 3차원 임베딩)
embeddings = np.array([[1, 2, 3],
                       [4, 5, 6],
                       [7, 8, 9],
                       [10, 11, 12]])

# 임베딩 차원 (임베딩 벡터의 길이)와 토큰 수 정의
embedding_dim = 3
num_tokens = embeddings.shape[0]

# 회전 각도를 설정 (임베딩 차원에 따라 각도를 다르게 설정할 수 있음)
theta = 0.1  # 예시로 작은 각도 설정

# 토큰마다 RoPE 적용
rotated_embeddings = []
for p in range(1, num_tokens + 1):
    rotated_embedding = apply_rotary_position_embedding(embeddings[p-1], p, theta)
    rotated_embeddings.append(rotated_embedding)

# 결과 출력
print("원래 임베딩 벡터:")
print(embeddings)
print("\nRoPE 적용 후 회전된 임베딩 벡터:")
print(np.array(rotated_embeddings))


원래 임베딩 벡터:
[[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10 11 12]]

RoPE 적용 후 회전된 임베딩 벡터:
[[ 0  2  3]
 [ 2  5  6]
 [ 4  9  9]
 [ 4 14 12]]
